<a href="https://colab.research.google.com/github/indl1670/KST_Project/blob/maskrcnn/detectron2_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/kst_project/oneCycle

/content/drive/MyDrive/kst_project/oneCycle


In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

In [4]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [5]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("coco2021_train", {}, "/content/drive/Shareddrives/KST_Project/polygon/annotations/instances_train2021.json",
                        "/content/drive/Shareddrives/KST_Project/polygon/images/train2021")
register_coco_instances("coco2021_val", {}, "/content/drive/Shareddrives/KST_Project/polygon/annotations/instances_val2021.json",
                        "/content/drive/Shareddrives/KST_Project/polygon/images/val2021")

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("coco2021_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [46]:
import json
import os
import shutil

train_dir = '/content/drive/Shareddrives/KST_Project/polygon/annotations/instances_train2021.json'
image_dir = '/content/drive/Shareddrives/KST_Project/polygon/images/train2021/'
with open(train_dir, 'r', encoding='utf8') as j:
  data = json.load(j)
  images = data['images']
  tmp = []
  print(len(images))
  for img in images:
    name = img['file_name']
    if not os.path.isfile(image_dir + name):
      tmp.append(name)

print(len(os.listdir(image_dir)))
print(len(tmp))
print(tmp)
# c = []
# for t in tmp:
#   if os.path.isfile('/content/drive/Shareddrives/KST_Project/all_images/' + t):
#     # print('in')
#     shutil.copy('/content/drive/Shareddrives/KST_Project/all_images/' + t, image_dir + t)

11300
11250
50
['PM_sunny_CI04_20210928_150524_585_1.jpg', 'PM_sunny_CI04_20210928_150524_584_3.jpg', 'PM_sunny_CI04_20210928_150524_584_2.jpg', 'PM_sunny_CI04_20210928_150524_584_1.jpg', 'PM_sunny_CI04_20210928_150524_583_3.jpg', 'PM_sunny_CI04_20210928_150524_583_2.jpg', 'PM_sunny_CI04_20210928_150524_583_1.jpg', 'PM_sunny_CI04_20210928_150524_582_3.jpg', 'PM_sunny_CI04_20210928_150524_582_2.jpg', 'PM_sunny_CI04_20210928_150524_582_1.jpg', 'PM_sunny_CI04_20210928_150524_581_3.jpg', 'PM_sunny_CI04_20210928_150524_581_2.jpg', 'PM_sunny_CI04_20210928_150524_581_1.jpg', 'PM_sunny_CI04_20210928_150524_580_3.jpg', 'PM_sunny_CI04_20210928_150524_580_2.jpg', 'PM_sunny_CI04_20210928_150524_580_1.jpg', 'PM_sunny_CI04_20210928_150524_579_3.jpg', 'PM_sunny_CI04_20210928_150524_579_2.jpg', 'PM_sunny_CI04_20210928_150524_579_1.jpg', 'PM_sunny_CI04_20210928_150524_578_3.jpg', 'PM_sunny_CI04_20210928_150524_578_2.jpg', 'PM_sunny_CI04_20210928_150524_578_1.jpg', 'PM_sunny_CI04_20210928_150524_577_3.j

In [53]:
from glob import glob

find_train = 'PM_sunny_CI04_20210928_150524_585_1.jpg'
find_test = 'PM_sunny_CI04_20210928_150524_381_3.jpg'

json_dir = '/content/drive/Shareddrives/KST_Project/JSON'

train_json = glob(os.path.join(json_dir, '*.json'))

for t in train_json:
  with open(t, 'r', encoding='utf8') as js:
    data_js = json.load(js)
    img_list = data_js['images']
    for img in img_list:
      name = img['file_name']
      if find_test == img['file_name']:
        print(t)
      if find_train == img['file_name']:
        print(t)


/content/drive/Shareddrives/KST_Project/JSON/2021.11.22_L034_2.json
/content/drive/Shareddrives/KST_Project/JSON/2021.11.22_L034_1.json
